In [1]:
import bs4
import sqlite3

In [14]:
CREATE_TABLE_GAMES = '''CREATE TABLE games (
    game_id TEXT NOT NULL PRIMARY KEY,
    event TEXT,
    season INTEGER,
    date TEXT,
    status TEXT,
    home_team TEXT,
    away_team TEXT,
    home_score INTEGER,
    away_score INTEGER,
    home_2PA INTEGER,
    away_2PA INTEGER,
    home_2PM INTEGER,
    away_2PM INTEGER,
    home_3PA INTEGER,
    away_3PA INTEGER,
    home_3PM INTEGER,
    away_3PM INTEGER,
    home_FTA INTEGER,
    away_FTA INTEGER,
    home_FTM INTEGER,
    away_FTM INTEGER,
    home_OREB INTEGER,
    away_OREB INTEGER,
    home_DREB INTEGER,
    away_DREB INTEGER,
    home_AST INTEGER,
    away_AST INTEGER,
    home_BLK INTEGER,
    away_BLK INTEGER,
    home_STL INTEGER,
    away_STL INTEGER,
    home_TOV INTEGER,
    away_TOV INTEGER,
    home_PF INTEGER,
    away_PF INTEGER,
    home_odds REAL,
    away_odds REAL)
    '''

COLUMNS = [
    'game_id',
    'event',
    'season',
    'date',
    'status',
    'home_team',
    'away_team',
    'home_score',
    'away_score',
    'home_2PA',
    'away_2PA',
    'home_2PM',
    'away_2PM',
    'home_3PA',
    'away_3PA',
    'home_3PM',
    'away_3PM',
    'home_FTA',
    'away_FTA',
    'home_FTM',
    'away_FTM',
    'home_OREB',
    'away_OREB',
    'home_DREB',
    'away_DREB',
    'home_AST',
    'away_AST',
    'home_BLK',
    'away_BLK',
    'home_STL',
    'away_STL',
    'home_TOV',
    'away_TOV',
    'home_PF',
    'away_PF',
    'home_odds',
    'away_odds',
]

INSERT_INTO_GAMES_DICT = {key: None for key in COLUMNS}

INSERT_INTO_GAMES = '''INSERT INTO games VALUES (
    :game_id,
    :event,
    :season,
    :date,
    :status,
    :home_team,
    :away_team,
    :home_score,
    :away_score,
    :home_2PA,
    :away_2PA,
    :home_2PM,
    :away_2PM,
    :home_3PA,
    :away_3PA,
    :home_3PM,
    :away_3PM,
    :home_FTA,
    :away_FTA,
    :home_FTM,
    :away_FTM,
    :home_OREB,
    :away_OREB,
    :home_DREB,
    :away_DREB,
    :home_AST,
    :away_AST,
    :home_BLK,
    :away_BLK,
    :home_STL,
    :away_STL,
    :home_TOV,
    :away_TOV,
    :home_PF,
    :away_PF,
    :home_odds,
    :away_odds)
    '''

In [15]:
with sqlite3.connect('data/games.db') as con:
    query = CREATE_TABLE_GAMES
    cur = con.cursor()
    cur.execute(query)

In [16]:
def get_values_summary_source(summary_source, values_dict):
    soup = bs4.BeautifulSoup(summary_source, 'html.parser')
        
    event = soup.find('a', {'href': '/basketball/usa/nba/'})
    if event is not None:
        values_dict['event'] = event.text

    date = soup.find('div', {'class': 'duelParticipant__startTime'})
    if date is not None:
        values_dict['date'] = date.text

    status = soup.find('div', {'class': 'detailScore__status'})
    if status is not None:
        values_dict['status'] = status.text

    home_values = soup.find_all('div', {'class': 'smh__home'})[1:]
    home_values = [value.text for value in home_values]
    if len(home_values) > 2:
        values_dict['home_team'] = home_values[0]
        values_dict['home_score'] = home_values[1]

    away_values = soup.find_all('div', {'class': 'smh__away'})[1:]
    away_values = [value.text for value in away_values]
    if len(away_values) > 2:
        values_dict['away_team'] = away_values[0]
        values_dict['away_score'] = away_values[1]

    home_odds = soup.find('span', {'class': 'cell o_1'})
    if home_odds is not None:
        values_dict['home_odds'] = home_odds.get_text(' ').split()[-1]

    away_odds = soup.find('span', {'class': 'cell o_2'})
    if away_odds is not None:
        values_dict['away_odds'] = away_odds.get_text(' ').split()[-1]

In [18]:
def get_values_stats_source(stats_source, values_dict):
    soup = bs4.BeautifulSoup(stats_source, 'html.parser')

    category_names = soup.find_all('div', {'class': 'statCategoryName'})
    home_values = soup.find_all('div', {'class': 'statHomeValue'})
    away_values = soup.find_all('div', {'class': 'statAwayValue'})

    category_names_dict = {
        '2-Point Field G. Attempted': '2PA',
        '2-Point Field Goals Made': '2PM',
        '3-Point Field G. Attempted': '3PA',
        '3-Point Field Goals Made': '3PM',
        'Free Throws Attempted': 'FTA',
        'Free Throws Made': 'FTM',
        'Offensive Rebounds': 'OREB',
        'Defensive Rebounds': 'DREB',
        'Assists': 'AST',
        'Blocks': 'BLK',
        'Steals': 'STL',
        'Turnovers': 'TOV',
        'Personal Fouls': 'PF',
    }
    for category, home_value, away_value in zip(category_names, home_values, away_values):
        category = category.text
        if category in category_names_dict.keys():
            home_value, away_value = home_value.text, away_value.text
            values_dict[f'home_{category_names_dict[category]}'] = home_value
            values_dict[f'away_{category_names_dict[category]}'] = away_value

In [17]:
def insert_row_db_games(game_id, season, summary_source, stats_source):
    values_dict = INSERT_INTO_GAMES_DICT
    values_dict['game_id'] = game_id
    values_dict['season'] = season
    get_values_summary_source(summary_source, values_dict)
    if stats_source is not None:
        get_values_stats_source(stats_source, values_dict)

    with sqlite3.connect('data/games.db') as con:
        query = INSERT_INTO_GAMES
        cur = con.cursor()
        cur.execute(query, values_dict)

In [19]:
for year in range(1993, 2022):
    with sqlite3.connect(f'data/sources/sources_{year}.db') as con:
        query = 'SELECT id, summary_source, stats_source FROM sources'
        cur = con.cursor()
        cur.execute(query)
        rows = cur.fetchall()

    for game_id, summary_source, stats_source in rows:
        insert_row_db_games(game_id, year, summary_source, stats_source)